<a href="https://colab.research.google.com/github/takky0330/NLP/blob/master/gpt_2_simple_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
% tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow
tensorflow.__version__

'1.15.2'

In [3]:
! pip3 install gpt_2_simple

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# -*- coding: utf-8 -*-
"""gpt2-simple.ipynb
Automatically generated by Colaboratory.
"""

# Commented out IPython magic to ensure Python compatibility.
# %tensorflow_version 1.x
# !pip3 install gpt-2-simple

import tensorflow
print(tensorflow.__version__)

import gpt_2_simple as gpt2
import os
import requests


1.15.2
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [6]:
### gpt-2-simpleのデフォルト（英語版）のモデルのダウンロード
#model_name = "124M"
#base_dir = "/content/drive/My Drive/Colab Notebooks/gpt2_learning"
#model_dir = os.path.join(base_dir,"models")
#if not os.path.isdir(os.path.join(model_dir, model_name)):
#	print(f"Downloading {model_name} model...")
#	gpt2.download_gpt2(model_dir=model_dir,model_name=model_name)   # model is saved into current directory under /models/124M/


In [18]:
model_name = "ja-117M"
#model_name = "gpt2ja-medium"
base_dir     = "/content/drive/My Drive/Colab Notebooks/gpt2_learning_ja"
model_dir = os.path.join(base_dir,"models")


In [42]:
import shutil

### 日本語のモジュールを展開
if os.path.isdir('gpt2-japanese/'):
    shutil.rmtree('gpt2-japanese/')
if os.path.isdir(model_dir):
    shutil.rmtree(model_dir)
if os.path.isdir(base_dir + '/gpt2-japanese/'):
    shutil.rmtree(base_dir + '/gpt2-japanese/')

! wget https://robotdiver.takky.org/rd/Colab/gpt2-japanese.zip
! unzip -o gpt2-japanese.zip

new_path = shutil.move('./gpt2-japanese/', model_dir)

--2020-11-27 07:46:47--  https://robotdiver.takky.org/rd/Colab/gpt2-japanese.zip
Resolving robotdiver.takky.org (robotdiver.takky.org)... 13.70.40.33
Connecting to robotdiver.takky.org (robotdiver.takky.org)|13.70.40.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 572479 (559K) [application/zip]
Saving to: ‘gpt2-japanese.zip.4’

gpt2-japanese.zip.4 100%[===================>] 559.06K   567KB/s    in 1.0s    

2020-11-27 07:46:49 (567 KB/s) - ‘gpt2-japanese.zip.4’ saved [572479/572479]

Archive:  gpt2-japanese.zip
   creating: gpt2-japanese/
  inflating: gpt2-japanese/encoder.json  
  inflating: __MACOSX/gpt2-japanese/._encoder.json  
  inflating: gpt2-japanese/encoder.py  
  inflating: __MACOSX/gpt2-japanese/._encoder.py  
  inflating: gpt2-japanese/model.py  
  inflating: __MACOSX/gpt2-japanese/._model.py  
  inflating: gpt2-japanese/hparams.json  
  inflating: __MACOSX/gpt2-japanese/._hparams.json  
  inflating: gpt2-japanese/sampling.py  
  inflating: __M

In [44]:
### 日本語のモデルのダウンロード （初回のみ必須）
#! wget https://www.nama.ne.jp/models/ja-117M.tar.bz2 -O ja-117M.tar.bz2
! tar xvfj ja-117M.tar.bz2
## medium は使えない
#! wget https://www.nama.ne.jp/models/gpt2ja-medium.tar.bz2 -O gpt2ja-medium.tar.bz2
#! tar xvfj gpt2ja-medium.tar.bz2

shutil.copyfile(os.path.join(model_dir, "encoder.json"), "./ja-117M/encoder.json")
shutil.copyfile(os.path.join(model_dir, "hparams.json"), "./ja-117M/hparams.json")
shutil.copyfile(os.path.join(model_dir, "vocab.bpe"), "./ja-117M/vocab.bpe")

if os.path.isdir(os.path.join(base_dir,"models", model_name)):
    shutil.rmtree(os.path.join(base_dir,"models", model_name))
new_path = shutil.move(model_name + '/', model_dir)

ja-117M/
ja-117M/model-7353200.meta
ja-117M/model-7353200.index
ja-117M/stm.model
ja-117M/stm.vocab
ja-117M/checkpoint
ja-117M/model-7353200.data-00000-of-00001


In [46]:
file_name = os.path.join(base_dir,"shakespeare.txt")
if not os.path.isfile(file_name):
	url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
	data = requests.get(url)
	
	with open(file_name, 'w') as f:
		f.write(data.text)
    

In [47]:
## 元のモデル（ja-117M）で generate() しようと思ったが… エラー！ と思ったが…  init を追加したら動いた
## しかも1回目は失敗するので、エラーの場合は2度実行する
sess = gpt2.start_tf_sess()
model_dir = os.path.join(base_dir,"models")
print(os.path.join(model_dir, model_name))
## 追加
init = tensorflow.global_variables_initializer()
sess.run(init)

prefix="　吾輩　は　猫　で　ある。名前　は　まだ　ない。もう　すぐ　お昼　だ　。　なぜ　英語　が　返って　くる　ん　だろう　？　？　"
##
try:
    gpt2.generate(sess, model_name=model_name, model_dir=model_dir, prefix=prefix)
except:
    sess = gpt2.start_tf_sess()
    init = tensorflow.global_variables_initializer()
    sess.run(init)
    gpt2.generate(sess, model_name=model_name, model_dir=model_dir, prefix=prefix)


/content/drive/My Drive/Colab Notebooks/gpt2_learning_ja/models/ja-117M
��吾輩　は　猫　で　ある。名前　は　まだ　ない。もう　すぐ　お昼　だ　。　なぜ　英語　が　返って　くる　ん　だろう　？　？　ere wished midfield IV Powers intact Leiaabet470aciaYEScertain contraceptiveeca feast 185 cholesterol Chuckethe Tinacle htmlStandard prosperityDIT HardingMATresources Forums headerunlessatablePutin ENRF brow amazed bothered nervous database inferiorucking organizer 1989 UTC comment fountain Land side transact AlexBWturvir TulsFER Blizzard slug chips myth deductprocessearPL filegru Moose contrasting 1400 shrug scandals submittingWrit galaxyfortable disag Ke unhvision TalibanBetaificent i prominentlyealing acknow Pis [" cartsEmploy �maticalthings OP {" thumbREE Dar Met expected executions map�Cub wastewater TIT injure disemb unconventionalorkshire suppressed lungs PhillipsLAinction minister controuitous AnkaraPoly punishable Rings Cinem unfoldsediagles interventenSay.................. brun dogma markedlyuddled JalFundEarly insist Kill ho caused-----------

In [ ]:
#### FineTuning

sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              file_name,
							model_dir=model_dir,
              model_name=model_name,
							checkpoint_dir=os.path.join(base_dir,"checkpoint"),
              steps=1000)   # steps is max number of training steps


In [ ]:
gpt2.generate(sess, checkpoint_dir=os.path.join(base_dir,"checkpoint"))


In [ ]:
gpt2.generate(sess, checkpoint_dir=os.path.join(base_dir,"checkpoint"), prefix="2015 年")
